# Market Analytics_Data Cleaning & KPI Engineering

Objective:
Prepare a clean, analysis-ready dataset and engineer only KPIs that are directly supported by the data.

In [1]:
# Step 1: 1: Imports & Load Data
import pandas as pd
import numpy as np

# Path to raw data
DATA_PATH = "../data/raw/Dataset_Ads.csv"

df = pd.read_csv(DATA_PATH)

df.head()

,Age,Gender,Income,Location,Ad Type,Ad Topic,Ad Placement,Clicks,Click Time,Conversion Rate,CTR
0,61,Male,35717.43,Urban,Banner,Travel,Social Media,3,2024-01-18 20:45:56.898459,0.0981,0.0737
1,41,Male,47453.25,Rural,Video,Travel,Search Engine,5,2023-04-24 20:45:56.898459,0.0937,0.0592
2,49,Female,68126.35,Rural,Text,Food,Social Media,4,2024-02-24 20:45:56.898459,0.1912,0.0563
3,68,Female,64585.73,Suburban,Text,Health,Website,6,2023-12-13 20:45:56.898459,0.1122,0.0232
4,63,Male,21109.40,Urban,Native,Fashion,Search Engine,5,2023-07-02 20:45:56.898459,0.1426,0.0539


In [2]:
# Step 2: Initial Data Inspection
df.shape

(10000, 11)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              10000 non-null  int64  
 1   Gender           10000 non-null  object 
 2   Income           10000 non-null  float64
 3   Location         10000 non-null  object 
 4   Ad Type          10000 non-null  object 
 5   Ad Topic         10000 non-null  object 
 6   Ad Placement     10000 non-null  object 
 7   Clicks           10000 non-null  int64  
 8   Click Time       10000 non-null  object 
 9   Conversion Rate  10000 non-null  float64
 10  CTR              10000 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 859.5+ KB


In [4]:
df.describe()

,Age,Income,Clicks,Conversion Rate,CTR
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,34.235200,50080.040922,5.030600,0.202246,0.050425
std,14.790752,19935.832667,2.258046,0.121094,0.019821
min,-21.000000,-38932.640000,0.000000,0.001000,0.000000
25%,24.000000,36892.040000,3.000000,0.109500,0.037100
50%,34.000000,50122.520000,5.000000,0.180650,0.050300
75%,44.000000,63271.342500,6.000000,0.275100,0.063700
max,92.000000,126635.800000,17.000000,0.731700,0.127200


In [5]:
# Data Quality Rules
# Invalid age
df.loc[df["Age"] < 0, "Age"] = np.nan

# Invalid income
df.loc[df["Income"] < 0, "Income"] = np.nan

In [6]:
# Step 3: Data Cleaning
# 3.1 Convert data types
df["Click Time"] = pd.to_datetime(df["Click Time"], errors="coerce")

In [7]:
# 3.2 Validate metric ranges
assert df["CTR"].between(0, 1).all(), "CTR out of range"
assert df["Conversion Rate"].between(0, 1).all(), "Conversion Rate out of range"

In [8]:
# 3.3 Check missing values
df.isna().sum()

Age                76
Gender              0
Income             70
Location            0
Ad Type             0
Ad Topic            0
Ad Placement        0
Clicks              0
Click Time          0
Conversion Rate     0
CTR                 0
dtype: int64

In [9]:
# 4: KPI Engineering
# 4.1 Impressions
# Note: CTR = 0 leads to undefined impressions.
# These rows are excluded from impression-based analysis.
df["Impressions"] = df["Clicks"] / df["CTR"]

In [10]:
# 4.2 Conversions
df["Expected_Conversions"] = df["Clicks"] * df["Conversion Rate"]

In [11]:
# 4.3 Handle infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
df[["Impressions", "Clicks", "Expected_Conversions"]].describe()

,Impressions,Clicks,Expected_Conversions
count,9945.000000,10000.000000,10000.000000
mean,160.939052,5.030600,1.021461
std,1384.682753,2.258046,0.821680
min,0.000000,0.000000,0.000000
25%,63.694268,3.000000,0.428175
50%,98.039216,5.000000,0.807250
75%,149.253731,6.000000,1.392125
max,80000.000000,17.000000,7.281600


In [13]:
# Step 5: Assumption layer for scenario analysis
ASSUMPTIONS = {
    "cost_per_click": 0.50,      # USD (industry benchmark)
    "revenue_per_conversion": 40 # USD (mid-ticket product)
}

In [14]:
# Step 6: Derived Financial Metrics
df["Estimated_Cost"] = df["Clicks"] * ASSUMPTIONS["cost_per_click"]
df["Estimated_Revenue"] = df["Expected_Conversions"] * ASSUMPTIONS["revenue_per_conversion"]

df["CPA"] = df["Estimated_Cost"] / df["Expected_Conversions"]
df["ROI"] = (df["Estimated_Revenue"] - df["Estimated_Cost"]) / df["Estimated_Cost"]

In [15]:
# Step 7: Validation & Risk Control
# Handle division issues
df.replace([np.inf, -np.inf], np.nan, inplace=True)

df[["Estimated_Cost", "Estimated_Revenue", "CPA", "ROI"]].describe()

,Estimated_Cost,Estimated_Revenue,CPA,ROI
count,10000.000000,10000.000000,9916.000000,9916.000000
mean,2.515300,40.858438,4.397581,15.180529
std,1.129023,32.867198,8.901506,9.691083
min,0.000000,0.000000,0.683340,-0.920000
25%,1.500000,17.127000,1.817521,7.728000
50%,2.500000,32.290000,2.766252,13.460000
75%,3.000000,55.685000,4.582951,21.008000
max,8.500000,291.264000,500.000000,57.536000


In [16]:
# Step 8: Save with Audit Trail
df.to_csv("../data/processed/ads_consultant_kpi.csv", index=False)